In [1]:
!pip install --quiet -U langchain langchain-openai
%pip install --quiet --upgrade langchain langchain-community langchain-chroma


Note: you may need to restart the kernel to use updated packages.


In [23]:
!pip install --quiet supabase
!pip install scikit-learn

   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.3 MB 8.4 MB/s eta 0:00:01
   ----------- ---------------------------- 2.6/9.3 MB 6.6 MB/s eta 0:00:02
   ---------------- ----------------------- 3.9/9.3 MB 6.5 MB/s eta 0:00:01
   ---------------------- ----------------- 5.2/9.3 MB 6.4 MB/s eta 0:00:01
   -------------------------- ------------- 6.0/9.3 MB 5.8 MB/s eta 0:00:01
   --------------------------------- ------ 7.9/9.3 MB 6.2 MB/s eta 0:00:01
   ---------------------------------------  9.2/9.3 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 9.3/9.3 MB 6.2 MB/s eta 0:00:00


In [26]:
import os
from supabase import create_client, Client

openai_api_key = os.getenv("OPENAI_API_KEY_1010")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

print(f"OpenAI API: {openai_api_key[:4]}********")
print(f"LangChain API Key: {langchain_api_key[:4]}********")

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["LANGCHAIN_PROJECT"] = "tech4impact_1010"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_TRACING_V2"] = "true"

SUPABASE_URL = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
SUPABASE_KEY = os.getenv("NEXT_PUBLIC_SUPABASE_SERVICE_KEY")
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)



OpenAI API: sk-p********
LangChain API Key: lsv2********


LangSmith Setup (LLM Logging)


In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello there! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e10ecef1-b8ca-4f25-9e85-725a6c1b049c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21})

Starts from now

In [39]:
import os
import json
import getpass
import logging 
# Set your OpenAI API key
#os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory



In [4]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [50]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

TAGS = [
    "고지혈증 약", "비타민", "스테로이드", "타이레놀",
    "프로바이오틱스", "오메가 3", "아르기닌", "유산균",
    "건선", "고지혈증", "감기", "탈모", "갱년기",
    "운동", "술자리", "식사 조절", "수면 패턴",
    "간 수치", "가족력", "영양제",
    "기타"  # "기타" means "Other"
]
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")  # Efficient embedding model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Chat-based LLM for tagging

MIN_LENGTH = 20

# Define Semantic Merging Function Using Embeddings
def merge_semantically(utterances, similarity_threshold=0.8):

    if not utterances:
        return []
    
    # Extract messages for embedding
    messages = [utt['msg'] for utt in utterances]
    
    # Retrieve embeddings using LangChain
    utterance_embeddings = embeddings.embed_documents(messages)
    
    merged = []
    current = utterances[0].copy()
    current_embedding = utterance_embeddings[0]
    
    for idx in range(1, len(utterances)):
        next_utt = utterances[idx]
        next_embedding = utterance_embeddings[idx]
        
        # Check if the speaker is the same
        if current['spk'] == next_utt['spk']:
            # Compute cosine similarity
            similarity = cosine_similarity(
                [current_embedding],
                [next_embedding]
            )[0][0]
            
            if similarity >= similarity_threshold:
                # Merge utterances if similarity threshold is met
                current['duration'] += next_utt['duration']
                current['msg'] += " " + next_utt['msg']
                # Optionally, update other metadata if needed
            else:
                # Add current utterance to merged list and reset
                merged.append(current)
                current = next_utt.copy()
                current_embedding = next_embedding
        else:
            # If speakers are different, add current to merged list and reset
            merged.append(current)
            current = next_utt.copy()
            current_embedding = next_embedding
    
    merged.append(current)
    return merged
def merge_short_utterances(merged_utterances, min_length=MIN_LENGTH):
    if not merged_utterances:
        return []
    
    adjusted_utterances = []
    current = merged_utterances[0].copy()
    
    for next_utt in merged_utterances[1:]:
        # Count words or characters in the next utterance
        word_count = len(next_utt['msg'].split())
        
        if word_count < min_length:
            # If the next utterance is too short, merge it with the current one
            current['duration'] += next_utt['duration']
            current['msg'] += " " + next_utt['msg']
        else:
            # Append the current utterance if it meets the minimum length
            adjusted_utterances.append(current)
            current = next_utt.copy()
    
    # Add the last segment
    adjusted_utterances.append(current)
    return adjusted_utterances


# Define Tagging Function Using LangChain's ChatOpenAI
from typing import List


def semantic_tag_utterances_batch(
    merged_utterances: List[dict],
    tags_list: List[str],
    batch_size: int = 5,
    window_size: int = 1
) -> List[dict]:
    """
    Assigns semantic tags to merged utterances using LangChain's LLMChain in batches.
    Each utterance can have multiple tags or none (defaults to '기타').
    
    Args:
        merged_utterances (List[dict]): List of merged utterance dictionaries.
        tags_list (List[str]): List of predefined tags.
        batch_size (int, optional): Number of utterances to process in one batch. Defaults to 5.
        window_size (int, optional): Number of previous and next utterances to include as context. Defaults to 1.
    
    Returns:
        List[dict]: List of merged utterance dictionaries with assigned tags.
    """
    def clean_ai_response(response: str) -> str:
        """
        Cleans the AI response by removing code block markers if present.

        Args:
            response (str): The raw response from the AI.

        Returns:
            str: The cleaned JSON string.
        """
        # Remove triple backticks and language specifier if present
        if response.startswith("```") and "json" in response:
            response = response.strip("```json").strip("```").strip()
        elif response.startswith("```") and "json" not in response:
            response = response.strip("```").strip()
        return response

    tagged_utterances = []
    total = len(merged_utterances)
    tags_str = ", ".join(tags_list[:-1]) + ", 기타"
        
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)  # Adjust model and parameters as needed
    prompt_template = """
    You are an AI assistant that assigns relevant tags to conversation fragments based on the provided list of tags.
    Each fragment can have multiple tags or none. If no tags apply, assign '기타'.
    Return the results as a JSON array where each element contains the 'fragment_id' and a list of assigned 'tags'.

    Example Response:
    [
        {{"fragment_id": 0, "tags": ["비타민", "운동"]}},
        {{"fragment_id": 1, "tags": ["기타"]}}
    ]

    Tags List:
    {tags_str}

    Conversation Context:
    {chat_history}

    Fragments to Tag:
    {fragments}

    Provide the assigned tags in JSON format as shown in the example.
    """

    # Create a PromptTemplate
    template = PromptTemplate(
        input_variables=["tags_str", "chat_history", "fragments"],
        template=prompt_template
    )


    # Create the LLMChain with the prompt and memory
    chain = LLMChain(
        llm=llm,
        prompt=template,
    )
    for i in range(0, total, batch_size):
        batch = merged_utterances[i:i+batch_size]
        
        # Determine the context window for the entire batch
        context_start = max(0, i - window_size)
        context_end = min(total, i + batch_size + window_size)
        context_utterances = merged_utterances[context_start:context_end]

        # Construct the conversation context
        conversation_context = ""
        for utt in context_utterances:
            speaker = f"Speaker {utt['spk']}"
            message = utt['msg']
            conversation_context += f"{speaker}: {message}\n"

        # Prepare fragments to tag with unique fragment_ids
        fragments_info = []
        for idx, utt in enumerate(batch):
            fragment_id = i + idx  # Unique identifier across the entire list
            fragments_info.append({
                "fragment_id": fragment_id,
                "fragment": utt['msg']
            })

        # Convert fragments_info to a JSON string for the prompt
        fragments_json = json.dumps([
            {"fragment_id": frag['fragment_id'], "fragment": frag['fragment']}
            for frag in fragments_info
        ], ensure_ascii=False, indent=4)

        # Prepare the prompt variables
        prompt_variables = {
            "tags_str": tags_str,
            "chat_history": conversation_context,
            "fragments": fragments_json
        }

        # Generate the prompt using the PromptTemplate
        prompt = template.format(**prompt_variables)

        try:
            # Run the LLMChain with the prompt
            response = chain.run(tags_str=tags_str, chat_history=conversation_context, fragments=fragments_json)
            
            # Parse the JSON response
            cleaned_response = clean_ai_response(response)
            tags_json = json.loads(cleaned_response)
            
            # Create a mapping from fragment_id to tags
            tags_mapping = {item['fragment_id']: item.get('tags', ["기타"]) for item in tags_json}
            
            for frag in fragments_info:
                assigned_tags = tags_mapping.get(frag['fragment_id'], ["기타"])
                # Ensure tags are valid
                valid_tags = [tag for tag in assigned_tags if tag in tags_list]
                if not valid_tags:
                    valid_tags = ["기타"]
                # Assign tags to the corresponding utterance
                utt = merged_utterances[frag['fragment_id']]
                utt['tags'] = valid_tags
                tagged_utterances.append(utt)
        
        except json.JSONDecodeError as parse_error:
            logging.error(f"Error decoding JSON response: {parse_error}")
            logging.error(f"AI Response: {response}")
            # Assign '기타' to all fragments in the batch in case of parse error
            for frag in fragments_info:
                utt = merged_utterances[frag['fragment_id']]
                utt['tags'] = ["기타"]
                tagged_utterances.append(utt)
        
        except Exception as e:
            logging.error(f"Error during semantic tagging: {e}")
            # Assign '기타' to all fragments in the batch in case of API call failure
            for frag in fragments_info:
                utt = merged_utterances[frag['fragment_id']]
                utt['tags'] = ["기타"]
                tagged_utterances.append(utt)

    return tagged_utterances

In [48]:

with open('../transcription.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

utterances = data['utterances']
merged_utterances = merge_semantically(utterances, similarity_threshold=0.81)
merged_utterances_short_removed =  merge_short_utterances(merged_utterances, min_length=MIN_LENGTH)
from datetime import datetime  
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f"merged_utterances_{timestamp}.json"

# Save the `tagged_utterances` to a local JSON file
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(merged_utterances, f, ensure_ascii=False, indent=4)

        # Step 2: Tagging Merged Utterances in Batches
        


2024-11-12 15:22:53,094 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [51]:
tagged_utterances = semantic_tag_utterances_batch(merged_utterances, TAGS, batch_size=5)


2024-11-12 15:26:27,205 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-12 15:26:28,849 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-12 15:26:30,718 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-12 15:26:32,881 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-12 15:26:35,051 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-12 15:26:36,710 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-12 15:26:38,626 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-12 15:26:40,764 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-12 15:26:43,291 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "

In [32]:
def ms_to_minutes_seconds_str(milliseconds):
    seconds = milliseconds // 1000  # 총 초
    minutes = seconds // 60  # 분 계산
    remaining_seconds = seconds % 60  # 남은 초 계산
    return f"{minutes}분 {remaining_seconds}초"  # 출력 형식

# 파일 
n = 3941239

# 발화를 Document 객체로 생성
filtered_documents = [
    Document(
        page_content=utterance['msg'],
        metadata={
            'start_at': utterance['start_at'],
            'duration': utterance['duration'],
            'speaker': utterance['spk'],
            'speaker_type': utterance['spk_type']
        }
    )
    for utterance in utterances if utterance['start_at'] > n
]
logging.info(f"총 {len(filtered_documents)}개의 문서 생성.")

# 시작 시간 기준으로 정렬
documents = sorted(filtered_documents, key=lambda doc: doc.metadata['start_at'])


2024-11-12 14:24:34,108 - INFO - 총 403개의 문서 생성.


In [10]:
full_text = ''
for doc in documents:
    start_time_str = ms_to_minutes_seconds_str(doc.metadata['start_at'])
    full_text += f"[{start_time_str}] {doc.page_content}\n"


# 언어 모델 초기화 # mini 버전 써보기
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)



# 임베딩 및 벡터스토어 초기화
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings, persist_directory=None)

# 리트리버 
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


C:\Users\yooji\AppData\Local\Temp\ipykernel_7124\454676864.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
C:\Users\yooji\AppData\Local\Temp\ipykernel_7124\454676864.py:13: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings()
2024-11-12 13:44:23,920 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for mor

In [ ]:
import re
# 역할 지정해서 돌려보기
#role description
#확정은 결과물 가지고 펠로우님이랑 이야기
#chunk 늘려보기..

prompt = """
        당신은 전문적인 상담 세션을 분석하고 요약하는 AI입니다. 아래 상담 세션의 전체 스크립트에서 중요한 키워드를 7~20개 추출해 주세요. 다음 지침을 반드시 따르세요:
        
        1. 약 이름, 환자의 질병, 생활 습관이 중요한 키워드 후보입니다. 이에 주의를 기울이세요. 
        2. 추출한 키워드를 의약품, 질병, 생활 습관, 그 외로 분류하세요.


        **답변 예시:**
        {{약: [타이레놀, 비타민C],
        질병: [기립성 저혈압],
        생활 습관: [수면 부족],
        그 외: []
        }}
        **스크립트:**
        {text}
        
        **요약:**
"""


human_prompt = PromptTemplate(
    input_variables=[],
    template="요약:"
)


rag_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",  # 또는 "map_reduce" 등 필요에 따라 변경 가능
            retriever=retriever,
            return_source_documents=True
        )

# 전체 스크립트 요약 
response = rag_chain({"query": prompt.format(text=full_text)})
session_summary = response["result"]
source_documents = response["source_documents"]

print("=== Session Summary ===")
print(session_summary)
# print("\n=== Source Documents ===")
# for doc in source_documents:
#     print(f"Source: {doc.page_content}\n")


2024-11-12 13:56:59,903 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-12 13:57:11,772 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Session Summary ===
{약: [고지혈증 약, 비타민, 스테로이드, 타이레놀, 프로바이오틱스, 오메가 3, 아르기닌, 유산균],
질병: [건선, 고지혈증, 감기, 탈모, 갱년기],
생활 습관: [운동, 술자리, 식사 조절, 수면 패턴],
그 외: [간 수치, 가족력, 영양제] 
}


In [11]:
# 출력 파싱, 주제와 시간 정보 추출
subtopics = re.findall(r'\*\*(\d+분 \d+초) ~ (\d+분 \d+초)\*\*\n- (.+?)(?=\n\*\*\d+분|\Z)', session_summary, re.DOTALL)

print(subtopics)


[('66분 8초', '66분 42초', 'Q. 고지혈증 약을 끊었는데, 영양제를 먹어야 할까요?\n'), ('67분 4초', '67분 19초', '[이렇게 하세요] 간 수치가 높지 않다면, 영양제를 드셔도 괜찮습니다.\n'), ('69분 10초', '69분 15초', '[이렇게 하세요] 피로감이 있다면, 영양제를 드시는 것이 좋습니다.\n'), ('69분 29초', '70분 40초', 'Q. 병원에서 처방받은 약과 약국에서 구입한 약의 차이는 무엇인가요?\n'), ('71분 5초', '71분 10초', '[바꾸세요] 감기 초기에 비타민을 섭취하여 면역력을 높이세요.\n'), ('72분 1초', '72분 17초', '[이렇게 하세요] 감기약은 증상에 맞춰서 드세요.\n'), ('74분 2초', '74분 22초', '[이렇게 하세요] 열이 나면 타이레놀을 드세요.\n'), ('87분 48초', '88분 26초', '[이렇게 하세요] 고지혈증 약을 오래 복용했다면 코엔자임 큐텐을 보충하세요.\n'), ('89분 10초', '89분 19초', '[이렇게 하세요] 저녁 식사를 간단히 하고 일찍 주무세요.\n'), ('92분 8초', '92분 50초', '[이렇게 하세요] 저녁 식사는 탄수화물보다 단백질과 채소 위주로 드세요.\n'), ('93분 0초', '93분 34초', '[바꾸세요] 술로 인해 잠을 자는 것은 피하세요.\n'), ('94분 8초', '94분 49초', '[이렇게 하세요] 잠들기 전에는 소음 없이 주무세요.\n'), ('97분 0초', '97분 11초', '[이렇게 하세요] 주말에는 낮잠을 통해 피로를 푸세요.\n'), ('98분 21초', '98분 34초', '[이렇게 하세요] 필요에 따라 영양제를 추천받아 드세요.\n'), ('99분 8초', '99분 17초', '[이렇게 하세요] 상담 후 맞춤형 영양제를 제공받으세요.')]


In [13]:
def time_str_to_ms(time_str):
    match = re.match(r'(\d+)분 (\d+)초', time_str)
    if match:
        minutes = int(match.group(1))
        seconds = int(match.group(2))
        return (minutes * 60 + seconds) * 1000
    else:
        logging.warning(f"시간 형식이 일치하지 않습니다: {time_str}")
        return 0
    
json_results = []
        
        # 주제별 루프 처리
for idx, (start_time_str, end_time_str, content) in enumerate(subtopics, 1):
    subtopic = content.strip()
    start_time_ms = time_str_to_ms(start_time_str)
    end_time_ms = time_str_to_ms(end_time_str)
    
    # 해당 시간 범위 내에서 retrieve
    retrieved_docs = retriever.get_relevant_documents(subtopic)
    
    relevant_docs = [
        doc for doc in retrieved_docs
        if start_time_ms <= doc.metadata['start_at'] <= end_time_ms
    ]
    
    # 관련 스크립트 생성
    related_scripts = [
        {
            "time": ms_to_minutes_seconds_str(doc.metadata['start_at']),
            "content": doc.page_content
        } for doc in relevant_docs
    ]
    
    # 주제별 데이터를 딕셔너리로 저장
    json_result = {
        "topic_id": idx,
        "start_time": start_time_str,
        "end_time": end_time_str,
        "content": subtopic,
        "related_scripts": related_scripts
    }
    json_results.append(json_result)
print(json_results)

C:\Users\yooji\AppData\Local\Temp\ipykernel_21116\3804599114.py:20: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retrieved_docs = retriever.get_relevant_documents(subtopic)
2024-11-08 15:27:57,718 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-08 15:27:57,990 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-08 15:27:58,475 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-08 15:27:58,841 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-08 15:27:59,465 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-08 15:27:59,955 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-08 15:28:00,507 - INFO - HTTP Request: POST https://a

[{'topic_id': 1, 'start_time': '66분 8초', 'end_time': '66분 42초', 'content': 'Q. 고지혈증 약을 끊었는데, 영양제를 먹어야 할까요?', 'related_scripts': [{'time': '66분 8초', 'content': '예, 보내주신 거는 미리 읽어보고 아는데요, 고지혈증 약만 드시고 계신 거죠.'}]}, {'topic_id': 2, 'start_time': '67분 4초', 'end_time': '67분 19초', 'content': '[이렇게 하세요] 간 수치가 높지 않다면, 영양제를 드셔도 괜찮습니다.', 'related_scripts': []}, {'topic_id': 3, 'start_time': '69분 10초', 'end_time': '69분 15초', 'content': '[이렇게 하세요] 피로감이 있다면, 영양제를 드시는 것이 좋습니다.', 'related_scripts': []}, {'topic_id': 4, 'start_time': '69분 29초', 'end_time': '70분 40초', 'content': 'Q. 병원에서 처방받은 약과 약국에서 구입한 약의 차이는 무엇인가요?', 'related_scripts': [{'time': '69분 54초', 'content': '근데 감기로만 비교를 하면 이제 그 약국에서 쓰는 약이나 병원, 약이나 차이는 그 한 가지예요.'}, {'time': '70분 28초', 'content': '예, 비슷한데 좀 그런 게 있고, 또 하나 차이는 병원에 가면 일단 직접 소독이 되잖아요.'}, {'time': '70분 21초', 'content': '되 그렇죠. 약 성분은 약국에서 그냥 파는 약이나 나머지 감기 뭐 콧물약은 비슷해요.'}, {'time': '69분 29초', 'content': '지금도 이제 간 게 기억 있는데 그냥 약국 가서 처약만 먹고 하거든요. 선생님, 근데 이게 병원에 가서 처방받는 게 나아요.'}]}, {'topic_

In [14]:
print(json_results[0])

{'topic_id': 1, 'start_time': '66분 8초', 'end_time': '66분 42초', 'content': 'Q. 고지혈증 약을 끊었는데, 영양제를 먹어야 할까요?', 'related_scripts': [{'time': '66분 8초', 'content': '예, 보내주신 거는 미리 읽어보고 아는데요, 고지혈증 약만 드시고 계신 거죠.'}]}


In [15]:
output_filename = 'json_results.json'

# Write json_results to a JSON file
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(json_results, f, ensure_ascii=False, indent=4)

print(f"JSON results saved to {output_filename}")

JSON results saved to json_results.json


Next steps:

to create docker and upload to lambda,
 docker buildx use mybuilder2
docker buildx build --platform linux/amd64 --load -t tenten_ai_topic .
docker tag tenten_ai_topic:latest 820604767531.dkr.ecr.ap-northeast-2.amazonaws.com/tenten_ai_topic:latest
docker push 820604767531.dkr.ecr.ap-northeast-2.amazonaws.com/tenten_ai_topic:latest

In [ ]:
If you use docker buildx, make sure to reuse the builder instance rather than creating a new one every time:
bash
Copy code
# Create the builder only once
docker buildx create --use --name mybuilder
Then, you can use the existing builder for subsequent builds:
bash
Copy code
docker buildx use mybuilder
docker buildx build --platform linux/amd64 --load -t tenten_ai_topic .
